In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import auc, precision_recall_curve, accuracy_score, precision_score, recall_score
import os
import glob
from src.helpers import trainer_factory

folder_path = os.getcwd() + "/data"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))


def eval_model(model, X_test, y_test):
    score = {}
    predictions = model.predict(X_test)

    predictions[predictions == -1] = 0

    precision, recall, _ = precision_recall_curve(y_test, predictions)
    score["auc"] = auc(recall, precision)
    score["precision"] = precision_score(y_test, predictions)
    score["recall"] = recall_score(y_test, predictions)
    score["accuracy"] = accuracy_score(y_test, predictions)

    return score

print("Setup Complete")

Setup Complete


In [2]:
# This is really ugly and needs to be cleaned up more elegantly
captures = pd.read_csv(csv_files[0])
filename = csv_files[0].split("/")[-1]
captures["Source File"] = filename

for file in csv_files[1:]:
    filename = file.split("/")[-1]
    df = pd.read_csv(file)
    df["Source File"] = filename
    pd.concat([captures, df])

In [3]:
captures.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [4]:
captures.rename(columns={name:name.strip() for name in captures.columns}, inplace=True)

In [5]:
captures.shape

(170366, 80)

In [6]:

captures["Label"].unique()


array(['BENIGN', 'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection'], dtype=object)

In [7]:
captures["Label"] = captures["Label"] == 'BENIGN'

In [8]:
captures.dropna(inplace=True)

In [9]:
le = LabelEncoder()

captures["Label"] = le.fit_transform(captures["Label"])

This Next Block is important to note. The Isolation forest cannot deal with numbers that cannot be stored in float32s in its current state. Those must be delt with.

In [10]:
captures = captures[captures["Flow Bytes/s"] != np.inf]

In [11]:
captures.shape

(170231, 80)

In [12]:
features = captures.columns.drop(["Label", "Source File"])

X = captures[features]
y = captures["Label"]

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

In [ ]:
isoforest_trainer = trainer_factory(IsolationForest, X_train, y_train)

In [ ]:
isoforest = isoforest_trainer(random_state=0)

eval_model(isoforest, X_test, y_test)

{'auc': 0.9928511810631273,
 'precision': 0.9867269060928606,
 'recall': 0.9201418642292678,
 'accuracy': 0.9089477888998543}

The OCSVM took about an hour to trian and did not perform half as well as the isolation forest.
{'auc': 0.996731548529342,
 'precision': 0.9997661037563736,
 'recall': 0.5087117966295345,
 'accuracy': 0.5148973166032238}

In [ ]:
# ocsvm_trainer = trainer_factory(OneClassSVM, X_train, y_train)

In [ ]:
# ocsvm, _ = ocsvm_trainer()

# eval_model(ocsvm, X_test, y_test)

{'auc': 0.996731548529342,
 'precision': 0.9997661037563736,
 'recall': 0.5087117966295345,
 'accuracy': 0.5148973166032238}